# **Capítulo 2: Novos passos com Python**
Neste capítulo, vamos aprofundar nossos conhecimentos, aprendendo a interagir com arquivos, criar visualizações de dados e manipular datas, que são tarefas essenciais para qualquer análise financeira.

## **2.1 Leitura de arquivos .csv**
A biblioteca **Pandas** é a ferramenta fundamental para ler dados tabulares (como planilhas) em Python. Para ler arquivos no formato `.csv`, usamos a função `pd.read_csv()`.

Por padrão, a função espera que as colunas sejam separadas por vírgula. Se o seu arquivo usar ponto e vírgula (comum em arquivos brasileiros), você pode especificar isso com o argumento `sep=';'`.

In [ ]:
import pandas as pd
import numpy as np

# O código abaixo tentará ler o arquivo "abev3_petr4.csv".
# Se o arquivo não for encontrado, ele criará um DataFrame de exemplo para que o resto do tutorial funcione.

try:
    # Tenta ler o arquivo do disco
    df = pd.read_csv("abev3_petr4.csv")
    print("Arquivo 'abev3_petr4.csv' lido com sucesso!")
except FileNotFoundError:
    print("Arquivo 'abev3_petr4.csv' não encontrado. Criando um DataFrame de exemplo.")
    # Criando dados de exemplo
    datas = pd.date_range(start='2020-01-01', periods=100)
    dados_exemplo = {
        'data': datas,
        'petr4': (np.random.randn(100) * 0.02 + 0.0005).cumsum() + 25,
        'abev3': (np.random.randn(100) * 0.01 + 0.0002).cumsum() + 15
    }
    df = pd.DataFrame(dados_exemplo)
    # Calculando os retornos para usar nas próximas seções
    df['retorno_diario_petr4'] = df['petr4'].pct_change()
    df['retorno_diario_abev3'] = df['abev3'].pct_change()

# Exibindo as 5 primeiras linhas do DataFrame para verificação
df.head()

## **2.2 Gráficos com Matplotlib e Seaborn**

**Matplotlib** é a biblioteca base para visualizações em Python, enquanto **Seaborn** é construída sobre ela, oferecendo uma interface mais simples e gráficos estatísticos mais atraentes. A sintaxe básica envolve chamar uma função de plotagem e passar o DataFrame e as colunas desejadas.

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define o estilo visual dos gráficos para um fundo branco com grades
sns.set_theme(style="whitegrid")

### **2.2.1 Gráfico de Dispersão ou Gráfico de Pontos**

Este gráfico é usado para visualizar a relação entre duas variáveis numéricas.

In [ ]:
# Cria o gráfico de dispersão usando a função scatterplot do Seaborn
sns.scatterplot(data=df, x='petr4', y='abev3')

# Adiciona um título ao gráfico e o exibe
plt.title('Dispersão: Preços PETR4 vs. ABEV3')
plt.show()

### **2.2.2 Gráfico de Linha**

A sintaxe para fazer um gráfico de linha é semelhante à do gráfico de pontos. Contudo, esse tipo de gráfico é mais usado quando a variável do eixo X é uma data, sendo perfeito para visualizar a evolução dos preços de um ativo ao longo do tempo.

In [ ]:
# Cria o gráfico de linha usando a função lineplot do Seaborn
sns.lineplot(data=df, x='data', y='petr4')

# Adiciona um título ao gráfico
plt.title('Evolução do Preço de PETR4')

# Rotaciona os rótulos do eixo X para melhor visualização
plt.xticks(rotation=45)

# Exibe o gráfico
plt.show()

### **2.2.3 Histogramas**

Os histogramas são utilizados para representar a distribuição de dados de uma variável quantitativa em intervalos contínuos. No mercado financeiro, uma aplicação comum de histogramas é analisar a distribuição de retornos diários de um ativo.

In [ ]:
# Cria o histograma dos retornos de PETR4
# O argumento 'bins' controla o número de barras (intervalos)
# kde=True adiciona uma linha de estimativa de densidade sobre o histograma
sns.histplot(data=df, x='retorno_diario_petr4', bins=30, kde=True)

# Adiciona um título e exibe o gráfico
plt.title('Distribuição dos Retornos Diários de PETR4')
plt.show()

## **2.3 Trabalhando com Datas**

Datas são um caso especial em análise de dados. O Pandas possui um tipo de dado específico para elas, o `datetime`, que é extremamente poderoso. A função `pd.to_datetime()` é a principal ferramenta para converter colunas de texto (ou outros formatos) para o tipo `datetime`, o que nos permite realizar cálculos e extrair informações com facilidade.

In [ ]:
# Vamos garantir que nossa coluna 'data' esteja no formato correto
# Primeiro, vemos o tipo de dado atual da coluna
print(f"Tipo de dado ANTES da conversão: {df['data'].dtype}")

# Agora, usamos pd.to_datetime para converter
df['data'] = pd.to_datetime(df['data'])

# Verificamos o tipo novamente para confirmar a mudança
print(f"Tipo de dado DEPOIS da conversão: {df['data'].dtype}")

### **2.3.1 Gerar uma Sequência de Datas**

Para criar um vetor (ou uma lista) de datas sequenciais, usamos a função `pd.date_range()`. Ela é muito flexível e permite especificar o início, o fim, o número de períodos e a frequência (diária, mensal, anual, etc.).

In [ ]:
# Gerar um vetor de datas diárias entre duas datas
datas_diarias = pd.date_range(start="2020-01-01", end="2020-01-20", freq="D")
print("Datas Diárias:")
print(datas_diarias)

# Gerar um vetor de datas mensais (pegando o primeiro dia de cada mês)
datas_mensais = pd.date_range(start="2020-01-01", end="2020-12-01", freq="MS")
print("\nDatas Mensais:")
print(datas_mensais)

# Gerar 7 datas, uma por semana, a partir de uma data inicial
datas_semanais = pd.date_range(start="2020-01-01", periods=7, freq="W")
print("\nDatas Semanais (fim de semana):")
print(datas_semanais)

## **2.4 Convertendo Textos (Strings) para Datas**

Muitas vezes, as datas em um arquivo de dados vêm como texto, e em formatos variados. Aqui no Brasil, por exemplo, usamos o formato `DD/MM/AAAA`.

A função `pd.to_datetime()` é inteligente e consegue interpretar a maioria dos formatos comuns. Para ajudá-la a entender formatos como o brasileiro, onde o dia vem antes do mês, usamos o argumento `dayfirst=True`. A função também consegue interpretar informações de horário (hora, minuto, segundo).

In [ ]:
# Lista de datas em formato texto, com diferentes separadores
datas_brasil = ["01/12/2019", "20/11/2018", "30-01-1990"]

# Convertendo para o formato datetime
# dayfirst=True informa ao Pandas que o primeiro número é o dia
datas_convertidas = pd.to_datetime(datas_brasil, dayfirst=True)
print("Datas convertidas:")
print(datas_convertidas)


# Exemplo com dados de horário (hora, minuto e segundo)
data_hora_texto = "30-09-2019 14:51:39"
data_hora_convertida = pd.to_datetime(data_hora_texto, dayfirst=True)

print("\nData e hora convertidas:")
print(data_hora_convertida)
print(f"Tipo do objeto: {type(data_hora_convertida)}")

## **2.5 Extrair Componentes de uma Data**

Uma vez que uma coluna está no formato `datetime`, podemos facilmente extrair seus componentes (ano, mês, dia, dia da semana, etc.) usando o acessador `.dt`. Ele funciona como um portal para dezenas de propriedades e métodos relacionados a datas.

In [ ]:
# Vamos usar o vetor de datas que convertemos na seção anterior
datas_texto = ["01/12/2019 13:51:15", "20/11/2018 00:00:00", "30/01/1990 08:00:00"]
datas_convertidas = pd.to_datetime(datas_texto, dayfirst=True)

# Para usar o acessador .dt, precisamos que os dados estejam em uma Series do Pandas
datas_series = pd.Series(datas_convertidas)

# Extraindo os componentes:
print(f"Ano: {datas_series.dt.year.to_list()}")
print(f"Mês: {datas_series.dt.month.to_list()}")
print(f"Dia: {datas_series.dt.day.to_list()}")
print(f"Hora: {datas_series.dt.hour.to_list()}")
print(f"Dia da semana (nome): {datas_series.dt.day_name(locale='pt_BR').to_list()}")
print(f"Trimestre: {datas_series.dt.quarter.to_list()}")